# 보험 약관 챗봇 에이전트 (RAG-약관)

In [17]:
# === BLK1: Install & Env Check (revised with auto-discovery) ===
%pip install -q langchain langchain-openai langchain-community langchain-text-splitters \
               faiss-cpu pymupdf tiktoken pydantic "unstructured[pdf]" pytesseract pillow \
               pdfminer.six pypdf python-dotenv

import os, sys, platform, shutil
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# --- 필수 키 점검 (OpenAI) ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env에 추가 후 다시 실행하세요.")

# --- 파일 경로 자동 탐색 ---
CANDIDATES = [
    Path("/mnt/data/DBIS.pdf"),
    Path("./policies/DBIS.pdf"),
    Path("./DBIS.pdf"),
]

found = None
for p in CANDIDATES:
    if p.exists():
        found = p
        break

# 그래도 못 찾으면 작업 디렉토리 아래에서 재귀 탐색
if not found:
    for root, dirs, files in os.walk(".", topdown=True):
        if "DBIS.pdf" in files:
            found = Path(root) / "DBIS.pdf"
            break

if not found:
    raise FileNotFoundError(
        "❌ DBIS.pdf를 찾지 못했습니다. 다음 위치 중 하나에 파일을 두고 다시 실행하세요:\n"
        " - ./policies/DBIS.pdf\n - ./DBIS.pdf\n - /mnt/data/DBIS.pdf"
    )

# ./policies 로 정규화
POLICY_DIR = Path("./policies")
POLICY_DIR.mkdir(parents=True, exist_ok=True)
POLICY_PATH = POLICY_DIR / "DBIS.pdf"

if found.resolve() != POLICY_PATH.resolve():
    try:
        shutil.copy2(found, POLICY_PATH)
        msg = f"📄 파일을 {found} → {POLICY_PATH} 로 복사했습니다."
    except Exception as e:
        # 복사가 실패해도 원본 경로를 그대로 사용
        POLICY_PATH = found
        msg = f"⚠️ policies로 복사 실패. 원본 경로를 그대로 사용합니다: {POLICY_PATH}\n사유: {e}"
else:
    msg = f"📄 파일이 이미 {POLICY_PATH} 에 있습니다."

print("✅ BLK1 OK | OpenAI Key 확인 완료")
print("OS:", platform.system(), platform.version())
print(msg)
print("사용 파일 경로:", POLICY_PATH.resolve())


Note: you may need to restart the kernel to use updated packages.
✅ BLK1 OK | OpenAI Key 확인 완료
OS: Windows 10.0.19045
📄 파일이 이미 policies\DBIS.pdf 에 있습니다.
사용 파일 경로: C:\Users\배성우\Desktop\TIL\LLM-agent\policies\DBIS.pdf



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
# === BLK2-MULTI: Load multiple PDFs (with manual OCR fallback) ===
from pathlib import Path
from io import BytesIO
from collections import defaultdict, Counter
import platform, os

import fitz  # PyMuPDF
import pytesseract
from PIL import Image

try:
    from langchain_core.documents import Document
except Exception:
    from langchain.schema import Document

# 0) (선택) 특정 파일만 제한하고 싶으면 아래 리스트에 파일명을 적으세요.
#    예) INCLUDE_FILES = ["db(2p).pdf", "DBIS.pdf"]
INCLUDE_FILES = None   # 또는 ["db(2p).pdf"]

# 1) Tesseract 경로 자동 설정 (Windows)
if platform.system() == "Windows":
    candidates = [
        os.getenv("TESSERACT_CMD"),
        r"C:\Program Files\Tesseract-OCR\tesseract.exe",
        r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe",
    ]
    for c in candidates:
        if c and Path(c).exists():
            pytesseract.pytesseract.tesseract_cmd = c
            print("🧭 Tesseract 경로:", c)
            break

# 2) MuPDF 텍스트 추출
def extract_text_mupdf(pdf_path: Path):
    out = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            txt = page.get_text("text") or ""
            out.append(Document(page_content=txt, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in out)
    return out, total_chars

# 3) 수동 OCR (페이지→이미지→Tesseract)
def ocr_pdf_with_pytesseract(pdf_path: Path, dpi=300, lang="kor+eng"):
    out = []
    with fitz.open(str(pdf_path)) as doc:
        for i, page in enumerate(doc):
            mat = fitz.Matrix(dpi/72, dpi/72)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.open(BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img, lang=lang)
            out.append(Document(page_content=text, metadata={"source": str(pdf_path), "page": i}))
    total_chars = sum(len((d.page_content or "").strip()) for d in out)
    return out, total_chars

# 4) 여러 PDF 로드 (부족하면 OCR로 자동 폴백)
def load_policies_multi(policy_dir="./policies", include_files=None, min_chars=1000, ocr_lang="kor+eng"):
    policy_dir = Path(policy_dir)
    all_paths = sorted(policy_dir.glob("*.pdf"))
    if include_files:
        name_set = {n.lower() for n in include_files}
        all_paths = [p for p in all_paths if p.name.lower() in name_set]
    if not all_paths:
        raise FileNotFoundError(f"❌ PDF가 없습니다: {policy_dir.resolve()} (필터: {include_files})")

    loaded_docs = []
    audit_rows = []
    for pdf in all_paths:
        mdocs, mchars = extract_text_mupdf(pdf)
        mode = "mupdf"
        if mchars < min_chars:  # 텍스트 너무 적으면 OCR 시도
            print(f"⚠️ {pdf.name}: 텍스트 부족({mchars}) → OCR 시도")
            odocs, ochars = ocr_pdf_with_pytesseract(pdf, dpi=300, lang=ocr_lang)
            if ochars > 0:
                mdocs, mchars, mode = odocs, ochars, "ocr"
            else:
                print(f"  → OCR도 실패(0 chars). 이 파일은 텍스트 인식 불가일 수 있습니다.")
        loaded_docs.extend(mdocs)
        audit_rows.append({"file": pdf.name, "pages": len(mdocs), "chars": mchars, "mode": mode})
    return loaded_docs, audit_rows

# 5) 실행
POLICY_DIR = Path("./policies")
docs, audit = load_policies_multi(POLICY_DIR, include_files=INCLUDE_FILES, min_chars=1000, ocr_lang="kor+eng")

# 6) 결과 요약/프리뷰
print("\n=== Loaded PDFs ===")
for row in audit:
    print(f"- {row['file']}: pages={row['pages']}, chars={row['chars']}, mode={row['mode']}")
print(f"총 문서 청크(페이지) 수: {len(docs)}")

# 프리뷰
def inspect_docs(docs, max_preview=5):
    exts = []
    empty = 0
    for d in docs:
        src = d.metadata.get("source") or ""
        exts.append(Path(src).suffix.lower())
        if not (d.page_content or "").strip():
            empty += 1
    print("빈/짧은 문서 수:", empty)
    print("확장자 분포:", Counter(exts))
    for i, d in enumerate(docs[:max_preview], start=1):
        src = Path(d.metadata.get("source") or "n/a").name
        page = d.metadata.get("page", "n/a")
        txt = (d.page_content or "").strip().replace("\n", " ")
        print(f"[샘플 {i}] {src} (p.{page}) :: {txt[:300]}{'...' if len(txt)>300 else ''}")

inspect_docs(docs)

print("\n✅ BLK2-MULTI OK | 여러 PDF 로딩 완료")
print("   포함 파일(일부):", sorted({Path(d.metadata.get('source','')).name for d in docs})[:10])

🧭 Tesseract 경로: C:\Program Files\Tesseract-OCR\tesseract.exe
⚠️ DBIS.pdf: 텍스트 부족(0) → OCR 시도

=== Loaded PDFs ===
- db(2p).pdf: pages=588, chars=1132833, mode=mupdf
- DBIS.pdf: pages=43, chars=95988, mode=ocr
총 문서 청크(페이지) 수: 631
빈/짧은 문서 수: 0
확장자 분포: Counter({'.pdf': 631})
[샘플 1] db(2p).pdf (p.0) :: 122 어려운 용어는 프로미라이프 용어사전 참고 ……………………………………   70 인용 법규는   약관에서 인용한 법규  참고 ……………………………………  1663 용어 정의 갱신형 일정 기간을 주기로 보험기간이 자동 갱신되는 형태를 말합니다. 이  특별약관을 갱신형으로 가입한 경우에는 보험증권에 갱신 주기를 기 재하여 드립니다. 예시) 3년/10년/20년만기 자동갱신 비갱신형 만기까지 갱신되지 않고 보장이 지속되는 형태를 말합니다. 예시) 80세/90세/100세 만기 또는 10/15/20년 만기 1 (건강고지)상해후유장해(20-...
[샘플 2] db(2p).pdf (p.1) :: 124 ② 2025.1.1 2025.7.1 2026.1.1 보험가입일 질병으로 오른쪽 눈의  교정시력이0.1이하 (지급률15%)인 상태 상해로 그 오른쪽 눈의  교정시력이 0.02이하가  된 경우(지급률 35%) → 장해지급률 35%에서 질병으로 인한 장해지급률 15%를 뺀     20%(=35 %-15%)에 해당하는 후유장해보험금을 지급 󰊸회사가 지급하여야 할 하나의 상해로 인한 후유장해보험금은 보험가입금액을 한도 로 합니다. 3. (특별약관의 소멸) 󰊱보험증권에 기재된 피보험자가 보험기간 중에 사망할 경우에 이 특별약관은 소멸 ...
[샘플 3] db(2p).pdf (p.2) :: 126 󰊱회사가 1.(보험금의 지급사유)의 상해80%이상후유장해보험금

In [35]:
# === BLK3-PATCH: Safe batched embeddings to avoid 300k-tokens-per-request ===
from typing import List
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# ---- 전제: BLK2-MULTI가 완료되어 docs 리스트가 존재해야 합니다.
try:
    assert isinstance(docs, list) and len(docs) > 0
except Exception:
    raise RuntimeError("docs가 없습니다. 먼저 BLK2-MULTI를 성공적으로 실행하세요.")

# ---- 1) 청크 파라미터 (권장값: 정책 PDF에 적합)
CHUNK_SIZE = 600     # 문자 기준, 너무 크게 잡지 말기
CHUNK_OVERLAP = 80
print(f"✂️ Split params -> chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP}")

splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
split_docs = splitter.split_documents(docs)
print(f"✅ Split done -> chunks={len(split_docs)}")

# ---- 2) 임베딩 배치 파라미터
EMBED_MODEL   = "text-embedding-3-small"  # 비용/속도/차원(1536) 밸런스 좋음
BATCH_DOCS    = 64                        # 64개 청크씩 나눠 호출(총합 토큰 안전)
print(f"🧠 Embedding model={EMBED_MODEL}, batch_docs={BATCH_DOCS}")

embeddings = OpenAIEmbeddings(model=EMBED_MODEL)

# ---- 3) 배치로 안전하게 벡터스토어 빌드
vs = None
for i in range(0, len(split_docs), BATCH_DOCS):
    batch = split_docs[i:i+BATCH_DOCS]
    texts = [d.page_content for d in batch]
    metas = [d.metadata for d in batch]

    if vs is None:
        vs = FAISS.from_texts(texts, embeddings, metadatas=metas)
    else:
        # add_texts는 내부에서 동일 임베딩 인스턴스를 사용해 소규모로 호출합니다.
        vs.add_texts(texts, metadatas=metas)

    if (i // BATCH_DOCS) % 5 == 0:  # 간헐적 진행 상황 표기
        print(f"  ... embedded {min(i+BATCH_DOCS, len(split_docs))} / {len(split_docs)} chunks")

if vs is None:
    raise RuntimeError("벡터스토어 생성 실패: split_docs가 비어있거나 텍스트가 없습니다.")

print("✅ Vectorstore build complete.")

# ---- 4) 리트리버 (LLM 압축 포함)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
base_retriever = vs.as_retriever(search_kwargs={"k": 8})  # 필요시 6으로 낮춰 토큰 절약
retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)
print("✅ Retriever ready (k=8 + compression)")

# ---- 5) 간단 동작 확인
sample = split_docs[0].page_content[:200].replace("\n"," ")
print("🔎 sample chunk:", sample, "...")


✂️ Split params -> chunk_size=600, overlap=80
✅ Split done -> chunks=2613
🧠 Embedding model=text-embedding-3-small, batch_docs=64
  ... embedded 64 / 2613 chunks
  ... embedded 384 / 2613 chunks
  ... embedded 704 / 2613 chunks
  ... embedded 1024 / 2613 chunks
  ... embedded 1344 / 2613 chunks
  ... embedded 1664 / 2613 chunks
  ... embedded 1984 / 2613 chunks
  ... embedded 2304 / 2613 chunks
  ... embedded 2613 / 2613 chunks
✅ Vectorstore build complete.
✅ Retriever ready (k=8 + compression)
🔎 sample chunk: 122 어려운 용어는 프로미라이프 용어사전 참고 ……………………………………   70 인용 법규는   약관에서 인용한 법규  참고 ……………………………………  1663 용어 정의 갱신형 일정 기간을 주기로 보험기간이 자동 갱신되는 형태를 말합니다. 이  특별약관을 갱신형으로 가입한 경우에는 보험증권에 갱신 주기를 기 재하여 드립니다. 예시) 3년/10년/20 ...


In [36]:
# === BLK4: Structured outputs (Summary / Grading / Surgery Guide) ===
from typing import List, Dict, Tuple, Optional
from collections import OrderedDict

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 안전 확인 (앞 블록에서 생성됨)
try:
    retriever, llm
except NameError as e:
    raise RuntimeError("retriever/llm가 없습니다. 블록 3을 먼저 실행하세요.") from e

# --------------------------
# 공통 헬퍼
# --------------------------
AGE_BUCKETS = [(0, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 200)]

def build_context_from_queries(queries: List[str], top_k: int = 4, max_snippets: int = 18, max_chars: int = 1000):
    """
    검색어 리스트로 동의어 확장 검색 → (source, page) 기준 중복 제거 → 문맥블록/인용목록 생성
    """
    from pathlib import Path
    bucket: OrderedDict[Tuple[str, int], str] = OrderedDict()
    for q in queries:
        hits = search_with_synonyms(q, top_k=top_k)
        for src, page, txt in hits:
            key = (src, -1 if page is None else int(page))
            if key not in bucket and txt:
                bucket[key] = txt[:max_chars]
            if len(bucket) >= max_snippets:
                break
        if len(bucket) >= max_snippets:
            break

    # 문맥 블록과 인용 리스트
    ctx_lines, citations = [], []
    for i, ((src, page), txt) in enumerate(bucket.items(), start=1):
        ctx_lines.append(f"[Doc {i}] {src} (p.{page})\n{txt}")
        citations.append(f"{src} p.{page}")
    context_block = "\n\n".join(ctx_lines) if ctx_lines else "관련 근거를 찾지 못했습니다."
    return context_block, sorted(set(citations))


# --------------------------
# (A) 약관 요약 ≤5장
# --------------------------
class PolicySummary(BaseModel):
    executive_summary: str = Field(..., description="핵심 요약(10~15문장)")
    coverage_overview: List[str] = Field(..., description="주요 보장 항목(10~20개)")
    exclusions_limitations: List[str] = Field(..., description="주요 면책/제한 사항(5~15개)")
    claim_process: List[str] = Field(..., description="청구 절차 요약(5~10단계)")
    caveats: List[str] = Field(..., description="주의/모호/확인 필요 사항")
    citations: List[str] = Field(..., description="참고 원문(파일명 p.xx)")

summary_parser = PydanticOutputParser(pydantic_object=PolicySummary)
summary_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 보험 약관 전문 요약자다. 과장/추측 금지, 반드시 원문 근거에 의존하라. "
     "결과는 최대 A4 5장 분량에 맞게 간결·비중복으로 요약한다. "
     "이 답변은 법률/의료/재정 자문이 아니며, 최종 판단은 보험사/전문가 상담이 필요함을 전제로 한다."),
    ("human",
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "요구사항:\n- 종합 요약(10~15문장)\n- 보장 항목 목록(10~20)\n- 면책/제한(5~15)\n- 청구 절차(5~10단계)\n- 주의/모호/확인 필요사항\n"
     "- 원문 인용 목록(파일명 p.xx)\n\n{format_instructions}")
]).partial(format_instructions=summary_parser.get_format_instructions())

def summarize_policy_structured() -> dict:
    queries = [
        "보장 내용 요약", "면책", "제외", "청구 절차", "가입/납입/갱신", "해지", "감액", "대기기간",
        "3대 질병", "정의", "특약", "면책 사유", "필요서류"
    ]
    context_block, cites = build_context_from_queries(queries, top_k=4, max_snippets=20, max_chars=1200)
    chain = summary_prompt | llm | summary_parser
    out = chain.invoke({"context": context_block})
    result = out.model_dump()
    result["citations"] = cites
    return result


# --------------------------
# (B) 약관 평가 0~10점
# --------------------------
class ScoreDetail(BaseModel):
    criterion: str
    score: float
    rationale: str
    citations: List[str]

class PolicyGrade(BaseModel):
    overall_score: float
    scale: str
    details: List[ScoreDetail]
    gaps: List[str]
    improvement_suggestions: List[str]
    notes: str

grade_parser = PydanticOutputParser(pydantic_object=PolicyGrade)
grade_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 보험 약관 품질을 0~10점으로 평가한다. "
     "평가축: ①연령대별 보험료 적합성 ②3대 질병 진단비 보장 적정성 ③불필요/중복 약관 여부. "
     "판단은 약관 원문 근거에 기반해야 하며, 근거 부족은 감점 및 '불확실' 표기. "
     "이 결과는 정보 제공 목적이며, 최종 판단은 보험사/전문가 상담이 필요하다."),
    ("human",
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "평가 기준:\n- 연령대 범주: {age_buckets}\n- 3대 질병(암/뇌졸중/급성심근경색)\n- 불필요/중복/모호 조항\n\n"
     "가중치: 동일 가중치(각 1/3)\n\n{format_instructions}")
]).partial(format_instructions=grade_parser.get_format_instructions())

def grade_policy_structured() -> dict:
    price_terms = ["보험료", "월 보험료", "연령", "나이", "보험료표", "납입", "가입연령", "갱신", "비갱신"]
    disease_terms = [
        "암", "악성신생물", "cancer",
        "뇌졸중", "stroke", "CVA",
        "급성심근경색", "AMI", "myocardial infarction"
    ]
    queries = price_terms + disease_terms + ["면책", "제외", "중복", "특약", "모호", "단서"]
    context_block, cites = build_context_from_queries(queries, top_k=3, max_snippets=22, max_chars=900)

    chain = grade_prompt | llm | grade_parser
    out = chain.invoke({"context": context_block, "age_buckets": AGE_BUCKETS})
    result = out.model_dump()
    # 전체 인용 요약을 notes에 병기
    note_tail = f"\n\n참고 근거(집계): {', '.join(cites[:30])}" if cites else ""
    result["notes"] = (result.get("notes") or "") + note_tail
    return result


# --------------------------
# (C) 수술별 보장/청구 안내
# --------------------------
class SurgeryGuide(BaseModel):
    surgery: str
    patient_age: Optional[int]
    matched_benefits: List[str]
    payout_estimates: List[str]
    waiting_or_exclusions: List[str]
    claim_steps: List[str]
    required_documents: List[str]
    caveats: List[str]
    citations: List[str]

surgery_parser = PydanticOutputParser(pydantic_object=SurgeryGuide)
surgery_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 특정 수술에 대해 약관에서 보장/면책/청구조건을 찾아 정리한다. "
     "금액/비율/기한/예외/대상 여부/의학적 정의를 가능하면 원문에 근거해 요약하고, 불확실하면 '불확실' 표기. "
     "이 답변은 정보 제공 목적이며, 최종 청구/적용은 보험사 심사/전문가 상담이 필요하다."),
    ("human",
     "수술명: {surgery}\n나이: {age}\n\n"
     "컨텍스트(약관 발췌):\n{context}\n\n"
     "요구사항:\n- 관련 보장 항목\n- 지급액/지급비율 추정(근거 포함)\n- 대기/면책/예외 조건\n- 청구 절차 단계\n- 필요서류\n- 주의/모호 사항\n\n"
     "{format_instructions}")
]).partial(format_instructions=surgery_parser.get_format_instructions())

def surgery_coverage_structured(surgery: str, age: Optional[int] = None) -> dict:
    # 동의어 확장 기반 쿼리 생성
    terms = expand_terms_with_synonyms(surgery)
    base_queries = []
    for t in terms:
        base_queries += [t, f"{t} 보장", f"{t} 수술비", f"{t} 진단비", f"{t} 청구", f"{t} 면책", f"{t} 대기기간", f"{t} 필요서류", f"{t} 정의"]
    context_block, cites = build_context_from_queries(base_queries, top_k=4, max_snippets=20, max_chars=900)

    chain = surgery_prompt | llm | surgery_parser
    out = chain.invoke({"surgery": surgery, "age": age, "context": context_block})
    result = out.model_dump()
    result["citations"] = cites
    return result


print("✅ BLK4 OK | 구조화 함수 3종 준비 완료 (summarize_policy_structured / grade_policy_structured / surgery_coverage_structured)")


✅ BLK4 OK | 구조화 함수 3종 준비 완료 (summarize_policy_structured / grade_policy_structured / surgery_coverage_structured)


In [37]:
# === BLK5: Wrap tools + Create Agent ===
from typing import Optional, List
from langchain_core.tools import tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# 안전 확인 (앞 블록에서 생성됨)
try:
    llm, retriever
    summarize_policy_structured
    grade_policy_structured
    surgery_coverage_structured
    search_with_synonyms
except NameError as e:
    raise RuntimeError("필수 객체/함수가 없습니다. 블록 3~4를 먼저 실행하세요.") from e

# --------------------------
# 1) 툴 래핑
# --------------------------
@tool("summarize_policy", return_direct=False)
def summarize_policy() -> dict:
    """
    업로드된 약관(DBIS.pdf)을 최대 5장 분량으로 구조화 요약합니다.
    """
    return summarize_policy_structured()

@tool("grade_policy", return_direct=False)
def grade_policy() -> dict:
    """
    약관을 0~10점으로 평가하고, 세부 근거와 개선 제안을 제공합니다.
    평가축: 연령대별 보험료 적합성 / 3대 질병 진단비 적정성 / 불필요·중복 조항 여부
    """
    return grade_policy_structured()

@tool("surgery_coverage", return_direct=False)
def surgery_coverage(surgery: str, age: Optional[int] = None) -> dict:
    """
    특정 수술에 대해 보장/면책/청구 절차/필요서류를 약관 근거와 함께 구조화해 반환합니다.
    """
    return surgery_coverage_structured(surgery=surgery, age=age)

@tool("knowledge_retriever", return_direct=False)
def knowledge_retriever(query: str, top_k: int = 6) -> str:
    """
    동의어 확장 검색으로 관련 조항 스니펫을 반환합니다. (파일명/페이지 포함)
    """
    hits = search_with_synonyms(query, top_k=top_k)
    if not hits:
        return "관련 근거를 찾지 못했습니다."
    lines = []
    for i, (src, page, txt) in enumerate(hits, start=1):
        lines.append(f"[Hit {i}] {src} (p.{page})\n{txt}")
    return "\n\n".join(lines)

tools = [summarize_policy, grade_policy, surgery_coverage, knowledge_retriever]

# --------------------------
# 2) 에이전트 프롬프트 & 메모리
# --------------------------
agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 한국어를 기본으로 답하는 보험 약관 분석 에이전트다. "
     "반드시 약관 원문(파일/페이지)에 근거해 답하고, 불확실하면 '불확실'이라고 명시한다. "
     "법률/의료/재정 자문이 아닌 정보 제공 목적임을 분명히 하고, 최종 판단과 청구는 보험사 심사/전문가 상담이 필요하다고 고지해라. "
     "사용자의 질문이 비 trivial 하거나 근거가 애매하면 다음 툴을 적극 사용하라: "
     "knowledge_retriever(조항 스니펫) / summarize_policy(요약) / grade_policy(평가) / surgery_coverage(수술별 안내)."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# --------------------------
# 3) 에이전트 생성
# --------------------------
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

print("✅ BLK5 OK | 에이전트 준비 완료")
print("   - Tools :", [t.name for t in tools])
print("   - Memory:", type(memory).__name__, "| chat_history size =", len(memory.chat_memory.messages))


✅ BLK5 OK | 에이전트 준비 완료
   - Tools : ['summarize_policy', 'grade_policy', 'surgery_coverage', 'knowledge_retriever']
   - Memory: ConversationBufferMemory | chat_history size = 0


In [38]:
# === BLK6: Run examples ===

# 6-1) 약관 요약 (≤5장 분량)
print("====== [Test 1] summarize_policy ======")
resp_summary = agent_executor.invoke({
    "input": "약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해."
})
resp_summary  # dict 내 result에 구조화 요약이 들어옵니다.


====== [Test 1] summarize_policy ======


> Entering new AgentExecutor chain...

Invoking: `summarize_policy` with `{}`


{'executive_summary': "이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다. '암'의 정의는 악성신생물 분류표에 따라 정해지며, 전암 상태는 보장에서 제외됩니다. 보험금 지급사유에 대한 합의가 이루어지지 않을 경우 제3자의 의견을 따를 수 있으며, 이 경우 의료비용은 보험사가 부담합니다. 특별약관의 해약환급금은 보통약관에 따르며, 특정 조건 하에 보험료 납입이 면제될 수 있습니다.", 'coverage_overview': ['위암 진단비', '위암 치료를 위한 1차 표적항암제 처방·투약', '상급종합병원 1인실 질병입원일당', '건강고지 2대 양성신생물 진단비', '소액암 진단비', '암 진단비 II (유사암 제외)', '독감 항바이러스제 치료비', '보험금 지급사유에 따른 의료비', '계약자 적립액 지급', '미경과 보험료 지급'], 'exclusions_limitations': ['표적항암약물허가치료 미해당 시 보장 제외', '전암 상태는 보장 제외', '보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름', '특별약관의 해약환급금 지급 제외', '보험금 지급하지 않는 사유에 해당하는 경우', '미경과 보험료 지급 조건 미충족 시 지급 제외', '갱신형 계약의 경우 특정 조건 하에 갱신되지 않음', '유사암에 대한 보장 제외', '정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외', '보험금 청구 시 필요한 서류 미제출 시 지급 제외'], 'claim_process': ['보험수익자는 청구서를 작성하여 제출해야 함.', '사고증명서(진단서, 진료비계산서 등)를 제출해야 함.', '신분증(정

{'input': '약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.',
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된 서류는 의료법에 따라 규정된 의료기관에서 발급된 것이어야 함.\n\n### 주의 사항\n-

In [40]:
# 6-2) 약관 품질 평가 (0~10점)
print("====== [Test 2] grade_policy ======")
resp_grade = agent_executor.invoke({
    "input": (
        "이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. "
        "평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, "
        "grade_policy 툴을 사용해."
    )
})
resp_grade


====== [Test 2] grade_policy ======


> Entering new AgentExecutor chain...

Invoking: `grade_policy` with `{}`


{'overall_score': 7.0, 'scale': '0-10', 'details': [{'criterion': '연령대별 보험료 적합성', 'score': 8.0, 'rationale': '보험료 납입지원금 및 지급금액이 연령대에 따라 적절하게 설정되어 있어, 다양한 연령대의 피보험자에게 적합한 보험료 구조를 제공하고 있음.', 'citations': ['Doc 2 (p.375)']}, {'criterion': '3대 질병 진단비 보장 적정성', 'score': 7.0, 'rationale': '암, 뇌졸중, 급성심근경색증에 대한 진단 및 보장 기준이 명확하게 정의되어 있으나, 특정 암(갑상선암, 기타피부암 등)의 제외 조항이 있어 보장 범위가 제한적임.', 'citations': ['Doc 8 (p.496)', 'Doc 21 (p.278)']}, {'criterion': '불필요/중복 약관 여부', 'score': 6.0, 'rationale': '약관 내 중복된 정의 및 조항이 존재하여 일부 혼란을 초래할 수 있음. 특히 암의 정의와 관련된 조항이 여러 번 반복되고 있음.', 'citations': ['Doc 9 (p.123)', 'Doc 14 (p.245)']}], 'gaps': ['특정 암에 대한 제외 조항이 있어 보장 범위가 제한적임.', '중복된 정의 및 조항이 있어 약관의 명확성이 떨어짐.'], 'improvement_suggestions': ['암의 정의 및 보장 범위를 명확히 하고, 중복된 조항을 통합하여 약관의 가독성을 높일 것.', '연령대별 보험료 조정 시, 더 많은 연령대에 대한 세분화된 옵션을 제공할 것.'], 'notes': '이 평가는 약관 원문에 기반하여 이루어졌으며, 최종 판단은 보험사 또는 전문가 상담이 필요함.\n\n

{'input': '이 약관이 전체적으로 잘 되어있는지 0~10점으로 평가해줘. 평가축은 연령대별 보험료 적합성/3대 질병 진단비/불필요·중복 조항이고, grade_policy 툴을 사용해.',
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된

In [41]:
# 6-3) 수술별 보장/청구 안내 (동의어 매칭 예시 1: 한글명)
print("====== [Test 3] surgery_coverage — 관상동맥 우회술 (45세) ======")
resp_surg1 = agent_executor.invoke({
    "input": "surgery_coverage 툴로 '관상동맥 우회술'을 받는다면(나이 45) 보장/면책/청구 단계/필요서류를 근거와 함께 정리해줘."
})
resp_surg1


====== [Test 3] surgery_coverage — 관상동맥 우회술 (45세) ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': '관상동맥 우회술', 'age': 45}`


{'surgery': '관상동맥 우회술', 'patient_age': 45, 'matched_benefits': ['허혈심장질환으로 진단확정 후 수술을 받은 경우 보험가입금액 지급', '수술회차에 따라 체증형 지급금액 적용'], 'payout_estimates': ['1회차 수술: 보험가입금액의 100%', '2회차 수술: 보험가입금액의 130%', '3회차 수술: 보험가입금액의 160%', '4회차 수술: 보험가입금액의 190%', '5회차 수술: 보험가입금액의 220%', '6회차 이상 수술: 보험가입금액의 220%'], 'waiting_or_exclusions': ['부검감정서상 사인이 허혈심장질환으로 확정되거나 추정되는 경우에만 보장', '의료기관에서 의사의 관리 하에 치료가 필요하다고 인정된 경우에 한함'], 'claim_steps': ['보험금 청구서 작성', '사고증명서 제출 (진단서, 진료비계산서 등)', '신분증 제출', '기타 필요한 서류 제출'], 'required_documents': ['청구서 (회사양식)', '사고증명서 (진단서, 진료비계산서 등)', '신분증 (정부기관 발행)', '기타 보험금 수령에 필요한 서류'], 'caveats': ['허혈심장질환의 정의 및 진단확정은 의료법에 따라 이루어져야 함', '의사 자격증을 가진 자에 의해 진단되어야 함'], 'citations': ['db(2p).pdf p.263', 'db(2p).pdf p.267', 'db(2p).pdf p.271', 'db(2p).pdf p.294', 'db(2p).pdf p.343', 'db(2p).pdf p.36', 'db(2p).pdf p.401', 'db(

{'input': "surgery_coverage 툴로 '관상동맥 우회술'을 받는다면(나이 45) 보장/면책/청구 단계/필요서류를 근거와 함께 정리해줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된 서류는 의료법에 따라 규정된 의료기관

In [42]:
# 6-4) 수술별 보장/청구 안내 (동의어 매칭 예시 2: 영문 약어 CABG)
print("====== [Test 4] surgery_coverage — CABG (관상동맥우회술 동의어) ======")
resp_surg2 = agent_executor.invoke({
    "input": "surgery_coverage 툴로 'CABG' 수술을 받는다면(나이 45) 해당 보장/청구 조건을 알려줘."
})
resp_surg2


====== [Test 4] surgery_coverage — CABG (관상동맥우회술 동의어) ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': 'CABG', 'age': 45}`


{'surgery': 'CABG', 'patient_age': 45, 'matched_benefits': ['허혈심장질환으로 진단확정 후 수술을 받은 경우 보험금 지급', '수술회차에 따라 체증형허혈심장질환수술비 지급'], 'payout_estimates': ['1회차 수술: 보험가입금액의 100%', '2회차 수술: 보험가입금액의 130%', '3회차 수술: 보험가입금액의 160%', '4회차 수술: 보험가입금액의 190%', '5회차 수술: 보험가입금액의 220%', '6회차 이상 수술: 보험가입금액의 220%'], 'waiting_or_exclusions': ['부검감정서상 사인이 허혈심장질환으로 확정되거나 추정되는 경우에만 보장', '의사에 의해 치료가 필요하다고 인정된 경우에 한함'], 'claim_steps': ['청구서 작성 (회사양식)', '사고증명서 제출 (진단서, 진료비계산서 등)', '신분증 제출 (정부기관 발행 신분증)', '기타 필요 서류 제출'], 'required_documents': ['청구서 (회사양식)', '사고증명서 (진단서, 진료비계산서 등)', '신분증 (주민등록증, 운전면허증 등)', '기타 보험금 수령에 필요한 서류'], 'caveats': ['의료기관에서 의사의 관리 하에 수술이 이루어져야 함', '허혈심장질환의 정의 및 진단확정이 필요'], 'citations': ['db(2p).pdf p.263', 'db(2p).pdf p.267', 'db(2p).pdf p.271', 'db(2p).pdf p.272', 'db(2p).pdf p.294', 'db(2p).pdf p.343', 'db(2p).pdf p.36', 'db

{'input': "surgery_coverage 툴로 'CABG' 수술을 받는다면(나이 45) 해당 보장/청구 조건을 알려줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된 서류는 의료법에 따라 규정된 의료기관에서 발급된 것이어야 함.

In [43]:
# 6-5) 특정 조항 스니펫 찾기 (예: 3대 질병 중 '급성심근경색' 진단비)
print("====== [Test 5] knowledge_retriever — 급성심근경색 진단비 ======")
resp_hits = agent_executor.invoke({
    "input": "knowledge_retriever 툴로 '급성심근경색 진단비' 관련 조항과 면책/예외를 찾아서 파일명/페이지와 함께 보여줘."
})
resp_hits


====== [Test 5] knowledge_retriever — 급성심근경색 진단비 ======


> Entering new AgentExecutor chain...

Invoking: `knowledge_retriever` with `{'query': '급성심근경색 진단비', 'top_k': 6}`


[Hit 1] db(2p).pdf (p.278)
4. (“급성심근경색증”의 정의 및 진단확정)
󰊱이 특별약관에 있어서 “급성심근경색증”이라 함은 「통합형 심장관련질환Ⅰ 분류표」
(【별표59】통합형 심장관련질환Ⅰ 분류표 참조)에서 정한 급성심근경색증, 후속심
근경색증, 급성심근경색증 후 특정 현존 합병증을 말합니다.
󰊲「급성심근경색증」의 진단확정은 의료법 제3조에서 정한 국내의 병원이나 의원 또는 
이와 동등하다고 회사가 인정하는 국외의 의료기관의 의사(치과의사 제외) 자격증
을 가진자에 의하여 내려져야 하며, 이 진단은 병력과 함께 심전도, 심장초음파, 
관상동맥(심장동맥)촬영술, 혈액중 심장효소검사 등을 기초로 하여야 합니다. 그러
나 피보험자가 사망하여 상기 검사방법을 진단의 기초로 할 수 없는 경우 다음 각 
호의 어느 하나에 해당하는 경우에는 진단확정이 있는 것으로 볼 수 있습니다.
   ① 보험기간 중 급성심근경색증으로 진단 또는 치료를 받고 있었음을 증명할 수 
있는 문서화된 기록 또는 증거가 있는 경우

[Hit 2] db(2p).pdf (p.272)
4. (“급성심근경색증(I21)”의 정의 및 진단확정)
󰊱이 특별약관에 있어서 “급성심근경색증(I21)”이라 함은 제8차 한국표준질병사인분류 I21(급성심근경색증)에 해당하는 질병을 말합니다.
󰊲“급성심근경색증(I21)”의 진단확정은 의료법 제3조에서 정한 국내의 병원이나 의원 또는 이와 동등하다고 회사가 인정하는 국외의 의료기관의 의사(치과의사 제외) 자격증을 가진 자에 의하여 내려져야 하며, 이 진단은 병력과 함께 심전도, 심장초음파, 관상동맥(심장동맥)촬영술, 혈액중 심장효소검사 등을 기초

{'input': "knowledge_retriever 툴로 '급성심근경색 진단비' 관련 조항과 면책/예외를 찾아서 파일명/페이지와 함께 보여줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된 서류는 의료법에 따라 규정된 의료기관에서 발

In [44]:
# 6-6) 자유 질문 (에이전트가 툴 선택)
print("====== [Test 6] free-form ======")
resp_free = agent_executor.invoke({
    "input": (
        "내가 앞으로 '백내장수술'을 받을 수도 있어. "
        "해당 수술에 대해 실제 청구 시 주의해야 할 조건이나 대기기간, "
        "필요서류를 약관 근거와 함께 정리해줘."
    )
})
resp_free


====== [Test 6] free-form ======


> Entering new AgentExecutor chain...

Invoking: `surgery_coverage` with `{'surgery': '백내장수술'}`


{'surgery': '백내장수술', 'patient_age': None, 'matched_benefits': ['레이저(Laser)에 의한 안구 수술 보장', '냉동응고에 의한 안구 수술 보장'], 'payout_estimates': ['불확실'], 'waiting_or_exclusions': ['불확실'], 'claim_steps': ['수술 후 청구서 제출', '의사의 진단서 첨부', '보험사 심사 대기'], 'required_documents': ['수술 청구서', '의사의 진단서', '수술 관련 영수증'], 'caveats': ['보장 범위는 보험 약관에 따라 다를 수 있음', '특정 조건에 따라 면책이 적용될 수 있음'], 'citations': ['db(2p).pdf p.95']}### 백내장 수술 청구 정보

#### 1. 보장 내용
- **보장 항목**:
  - 레이저(Laser)에 의한 안구 수술 보장
  - 냉동응고에 의한 안구 수술 보장

#### 2. 청구 시 주의해야 할 조건
- **보장 범위**: 보장 범위는 보험 약관에 따라 다를 수 있으므로, 약관을 반드시 확인해야 합니다.
- **면책 조건**: 특정 조건에 따라 면책이 적용될 수 있으므로, 수술 전 보험사에 문의하여 확인하는 것이 좋습니다.

#### 3. 대기 기간
- **대기 기간**: 대기 기간에 대한 정보는 불확실하므로, 보험사에 직접 문의하여 확인해야 합니다.

#### 4. 필요 서류
- **수술 청구서**: 수술 후 작성하여 제출해야 합니다.
- **의사의 진단서**: 수술의 필요성을 증명하는 의사의 진단서가 필요합니다.
- **수술 관련 영수증**: 수술 비용을 증명할 수 있는 영수증을 제출해야 합니다.

#### 5. 청구 절차

{'input': "내가 앞으로 '백내장수술'을 받을 수도 있어. 해당 수술에 대해 실제 청구 시 주의해야 할 조건이나 대기기간, 필요서류를 약관 근거와 함께 정리해줘.",
 'chat_history': [HumanMessage(content='약관 전체 핵심을 요약해줘. 반드시 summarize_policy 툴을 사용해.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='약관의 핵심 요약은 다음과 같습니다:\n\n### 개요\n이 특별약관은 위암 진단 및 치료를 위한 1차 표적항암제의 처방과 투약에 대한 보장을 규정하고 있습니다. 그러나 표적항암약물허가치료에 해당하지 않는 경우 보장이 제외됩니다.\n\n### 보장 내용\n- 위암 진단비\n- 위암 치료를 위한 1차 표적항암제 처방·투약\n- 상급종합병원 1인실 질병입원일당\n- 건강고지 2대 양성신생물 진단비\n- 소액암 진단비\n- 암 진단비 II (유사암 제외)\n- 독감 항바이러스제 치료비\n- 보험금 지급사유에 따른 의료비\n- 계약자 적립액 지급\n- 미경과 보험료 지급\n\n### 제외 및 제한 사항\n- 표적항암약물허가치료 미해당 시 보장 제외\n- 전암 상태는 보장 제외\n- 보험금 지급사유에 대한 합의 불가 시 제3자의 의견에 따름\n- 특별약관의 해약환급금 지급 제외\n- 보험금 지급하지 않는 사유에 해당하는 경우\n- 미경과 보험료 지급 조건 미충족 시 지급 제외\n- 갱신형 계약의 경우 특정 조건 하에 갱신되지 않음\n- 유사암에 대한 보장 제외\n- 정당한 이유 없이 의사의 지시를 따르지 않은 경우 지급 제외\n- 보험금 청구 시 필요한 서류 미제출 시 지급 제외\n\n### 청구 절차\n1. 보험수익자는 청구서를 작성하여 제출해야 함.\n2. 사고증명서(진단서, 진료비계산서 등)를 제출해야 함.\n3. 신분증(정부기관 발행) 제출 필요.\n4. 기타 필요한 서류를 제출해야 함.\n5. 제출된 서류는 의료법에 따라

In [45]:
# === BLK7: Markdown formatters + save helpers ===
from pathlib import Path
from textwrap import indent
from IPython.display import Markdown, display

# -------- 공통 유틸 --------
def _h(level:int, text:str) -> str:
    return f"{'#'*level} {text.strip()}"

def _bullets(items, level=0) -> str:
    if not items: 
        return ""
    pad = "  " * level
    return "\n".join(f"{pad}- {str(x).strip()}" for x in items)

def _save_md(md_text:str, path:str) -> str:
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    # Windows 메모장 호환을 위해 utf-8-sig 권장
    p.write_text(md_text, encoding="utf-8-sig")
    return str(p.resolve())

def _show(md_text:str):
    display(Markdown(md_text))

# -------- (A) 약관 요약 렌더러 --------
def render_summary_md(data:dict, title:str="보험약관 핵심 요약") -> str:
    ex = data.get("executive_summary", "")
    cov = data.get("coverage_overview", [])
    excl = data.get("exclusions_limitations", [])
    claim = data.get("claim_process", [])
    cave = data.get("caveats", [])
    cites = data.get("citations", [])

    parts = []
    parts.append(_h(1, title))
    parts.append(_h(2, "1) Executive Summary"))
    parts.append(ex.strip() if isinstance(ex, str) else str(ex))

    parts.append(_h(2, "2) 주요 보장 항목"))
    parts.append(_bullets(cov))

    parts.append(_h(2, "3) 면책/제한 사항"))
    parts.append(_bullets(excl))

    parts.append(_h(2, "4) 청구 절차 (요약)"))
    parts.append(_bullets(claim))

    parts.append(_h(2, "5) 주의/모호/확인 필요 사항"))
    parts.append(_bullets(cave))

    parts.append(_h(3, "참고 근거(파일 p.xx)"))
    parts.append(_bullets(cites))

    return "\n\n".join(parts).strip() + "\n"

def print_summary_md(data:dict, title="보험약관 핵심 요약"):
    md = render_summary_md(data, title=title)
    _show(md)
    return md

def save_summary_md(data:dict, path:str="./export/summary.md", title="보험약관 핵심 요약") -> str:
    md = render_summary_md(data, title=title)
    return _save_md(md, path)

# -------- (B) 약관 평가 렌더러 --------
def render_grade_md(data:dict, title:str="보험약관 품질 평가 (0~10점)") -> str:
    overall = data.get("overall_score", "")
    scale = data.get("scale", "")
    details = data.get("details", [])  # list of {criterion, score, rationale, citations}
    gaps = data.get("gaps", [])
    suggestions = data.get("improvement_suggestions", [])
    notes = data.get("notes", "")

    parts = []
    parts.append(_h(1, title))
    parts.append(f"**종합 점수:** {overall} / 10")
    if scale:
        parts.append(f"**척도/기준:** {scale}")

    parts.append(_h(2, "세부 평가"))
    if details:
        for i, d in enumerate(details, start=1):
            crit = d.get("criterion", "")
            sc = d.get("score", "")
            rat = d.get("rationale", "")
            cit = d.get("citations", [])
            parts.append(_h(3, f"{i}. {crit} — {sc}점"))
            parts.append(rat.strip())
            if cit:
                parts.append(_bullets([f"{c}" for c in cit]))
    else:
        parts.append("_세부 평가 항목이 없습니다._")

    parts.append(_h(2, "확인된 공백/리스크"))
    parts.append(_bullets(gaps) or "_(없음)_")

    parts.append(_h(2, "개선 제안"))
    parts.append(_bullets(suggestions) or "_(없음)_")

    if notes:
        parts.append(_h(3, "비고"))
        parts.append(notes.strip())

    return "\n\n".join(parts).strip() + "\n"

def print_grade_md(data:dict, title="보험약관 품질 평가 (0~10점)"):
    md = render_grade_md(data, title=title)
    _show(md)
    return md

def save_grade_md(data:dict, path:str="./export/grade.md", title="보험약관 품질 평가 (0~10점)") -> str:
    md = render_grade_md(data, title=title)
    return _save_md(md, path)

# -------- (C) 수술/질병 안내 렌더러 --------
def render_surgery_md(data:dict, title_prefix:str="수술·질병 보장/청구 안내") -> str:
    surgery = data.get("surgery", "")
    age = data.get("patient_age", None)
    benefits = data.get("matched_benefits", [])
    payouts = data.get("payout_estimates", [])
    waits = data.get("waiting_or_exclusions", [])
    steps = data.get("claim_steps", [])
    reqs = data.get("required_documents", [])
    cave = data.get("caveats", [])
    cites = data.get("citations", [])

    title = f"{title_prefix} — {surgery}" if surgery else title_prefix
    parts = []
    parts.append(_h(1, title))
    if age is not None:
        parts.append(f"**환자 나이:** {age}세")

    parts.append(_h(2, "관련 보장 항목"))
    parts.append(_bullets(benefits) or "_(해당 항목 없음)_")

    parts.append(_h(2, "지급액/지급비율 (근거 포함)"))
    parts.append(_bullets(payouts) or "_(명시된 지급 기준 없음)_")

    parts.append(_h(2, "대기기간/면책/예외"))
    parts.append(_bullets(waits) or "_(명시된 조건 없음)_")

    parts.append(_h(2, "청구 절차"))
    parts.append(_bullets(steps) or "_(미기재)_")

    parts.append(_h(2, "필요 서류"))
    parts.append(_bullets(reqs) or "_(미기재)_")

    parts.append(_h(2, "주의/모호 사항"))
    parts.append(_bullets(cave) or "_(없음)_")

    parts.append(_h(3, "참고 근거(파일 p.xx)"))
    parts.append(_bullets(cites))
    return "\n\n".join(parts).strip() + "\n"

def print_surgery_md(data:dict, title_prefix="수술·질병 보장/청구 안내"):
    md = render_surgery_md(data, title_prefix=title_prefix)
    _show(md)
    return md

def save_surgery_md(data:dict, path:str="./export/surgery.md", title_prefix="수술·질병 보장/청구 안내") -> str:
    md = render_surgery_md(data, title_prefix=title_prefix)
    return _save_md(md, path)

print("✅ BLK7 OK | Markdown 포맷/저장 헬퍼 준비 완료")

✅ BLK7 OK | Markdown 포맷/저장 헬퍼 준비 완료


In [46]:
# === BLK8: Build single bundled Markdown (summary + grade + surgery) ===
from datetime import datetime
from pathlib import Path

# 안전 체크
try:
    summarize_policy_structured, grade_policy_structured, surgery_coverage_structured
    render_summary_md, render_grade_md, render_surgery_md, _save_md
    POLICY_PATH
except NameError as e:
    raise RuntimeError("필수 함수/변수가 없습니다. BLK1~BLK7 순서대로 실행했는지 확인하세요.") from e

# ---- 사용자 입력(필요 시 수정) ----
SURGERY_NAME = "관상동맥 우회술"   # 예: "CABG", "백내장수술" 등으로 변경 가능
PATIENT_AGE  = 45

# ---- 결과 생성 ----
sum_data   = summarize_policy_structured()
grade_data = grade_policy_structured()
surg_data  = surgery_coverage_structured(SURGERY_NAME, age=PATIENT_AGE)

# ---- 섹션 렌더링 (개별 MD 생성) ----
md_summary = render_summary_md(sum_data, title="1) 약관 핵심 요약")
md_grade   = render_grade_md(grade_data, title="2) 약관 품질 평가 (0~10점)")
md_surgery = render_surgery_md(surg_data, title_prefix=f"3) 수술·질병 보장/청구 안내 — {SURGERY_NAME}")

# ---- 헤더/목차/면책 문구 ----
ts     = datetime.now().strftime("%Y-%m-%d %H:%M")
fname  = Path(POLICY_PATH).name if 'POLICY_PATH' in globals() else "DBIS.pdf"

header = f"""# DBIS 약관 분석 합본

- **원본 파일:** {fname}
- **생성 시각:** {ts}

> ⚠️ 면책 고지: 본 문서는 정보 제공 목적이며 법률/의료/재정 자문이 아닙니다. 실제 보장/지급 여부와 금액은 보험사 심사 및 약관 원문, 개별 특약, 청구 서류 등에 따라 달라질 수 있습니다. 최종 판단은 반드시 보험사/설계사/전문가와 상담하세요.

## 목차
1. 약관 핵심 요약
2. 약관 품질 평가 (0~10점)
3. 수술·질병 보장/청구 안내 — {SURGERY_NAME} ({PATIENT_AGE}세)
"""

# ---- 합본 조립 ----
bundle_md = "\n\n".join([header.strip(), md_summary.strip(), md_grade.strip(), md_surgery.strip()]) + "\n"

# ---- 저장 ----
out_path = "./export/DBIS_bundle.md"
saved = _save_md(bundle_md, out_path)

print("✅ 합본 저장 완료 →", saved)
print("💡 노션/깃허브에 그대로 붙여넣을 수 있습니다.")

✅ 합본 저장 완료 → C:\Users\배성우\Desktop\TIL\LLM-agent\export\DBIS_bundle.md
💡 노션/깃허브에 그대로 붙여넣을 수 있습니다.


In [ ]:
# === BLK9: Evidence Coverage Audit (premium/age, 3 diseases, exclusions/claims) ===
import re
import pandas as pd
from collections import defaultdict
from pathlib import Path

try:
    docs  # from previous blocks
except NameError:
    raise RuntimeError("docs 변수가 없습니다. 이전 블록을 먼저 실행하세요.")

# 1) 패턴/키워드
PREMIUM_TERMS = [r"보험료", r"월\s*보험료", r"연령", r"나이", r"세\b", r"가입\s*연령", r"납입", r"갱신", r"비갱신", r"보험료표"]
DISEASE_TERMS = sum([
    [r"암", r"악성신생물", r"cancer", r"malignant\s+neoplasm"],
    [r"뇌졸중", r"뇌졸증", r"중풍", r"stroke", r"cerebrovascular\s+accident", r"\bCVA\b"],
    [r"급성\s*심근경색", r"심근경색", r"acute\s+myocardial\s+infarction", r"\bAMI\b", r"myocardial\s+infarction"],
], [])
CLAIM_TERMS   = [r"청구", r"필요\s*서류", r"지급", r"면책", r"제외", r"대기\s*기간", r"보장하지\s*않는", r"제외\s*사항", r"예외"]

MONEY_PAT     = r"(\d{1,3}(?:[,\.]\d{3})+|\d+)\s*(원|만원|억|%)"
AGE_PAT       = r"(\d{1,3})\s*세"

def _count_hits(text:str, patterns) -> int:
    return sum(len(re.findall(p, text, flags=re.IGNORECASE)) for p in patterns)

def _cooccurrence_window(text:str, termsA, termsB, window=120):
    # termsA와 termsB가 근접(윈도우 내) 등장하는 횟수
    hits = 0
    for ta in termsA:
        for m in re.finditer(ta, text, flags=re.IGNORECASE):
            start = max(0, m.start()-window)
            end   = min(len(text), m.end()+window)
            seg = text[start:end]
            if any(re.search(tb, seg, flags=re.IGNORECASE) for tb in termsB):
                hits += 1
    return hits

rows = []
for d in docs:
    page = d.metadata.get("page", -1)
    txt = (d.page_content or "").strip()
    if not txt:
        continue
    # 기본 카운트
    prem_hits = _count_hits(txt, PREMIUM_TERMS)
    dis_hits  = _count_hits(txt, DISEASE_TERMS)
    clm_hits  = _count_hits(txt, CLAIM_TERMS)
    money_cnt = len(re.findall(MONEY_PAT, txt))
    age_cnt   = len(re.findall(AGE_PAT, txt))

    # 공존성(질병 용어 근처에 보장/지급/청구 등)
    cooc_benefit = _cooccurrence_window(txt, DISEASE_TERMS, [r"보장", r"진단비", r"수술비", r"지급"])
    cooc_claim   = _cooccurrence_window(txt, DISEASE_TERMS, [r"청구", r"필요\s*서류", r"면책", r"대기\s*기간"])

    rows.append({
        "page": page,
        "len": len(txt),
        "premium_hits": prem_hits,
        "disease_hits": dis_hits,
        "claim_hits": clm_hits,
        "money_tokens": money_cnt,
        "age_tokens": age_cnt,
        "cooc_benefit": cooc_benefit,
        "cooc_claim": cooc_claim,
    })

df = pd.DataFrame(rows).sort_values(["premium_hits","disease_hits","claim_hits","len"], ascending=False)
summary = {
    "pages_scanned": len(df),
    "pages_with_premium": int((df["premium_hits"]>0).sum()),
    "pages_with_disease": int((df["disease_hits"]>0).sum()),
    "pages_with_claim":   int((df["claim_hits"]>0).sum()),
    "pages_with_money":   int((df["money_tokens"]>0).sum()),
    "pages_with_age":     int((df["age_tokens"]>0).sum()),
    "cooc_benefit_total": int(df["cooc_benefit"].sum()),
    "cooc_claim_total":   int(df["cooc_claim"].sum()),
}
print("=== Evidence Coverage Summary ===")
for k,v in summary.items():
    print(f"{k}: {v}")

print("\n=== Top pages (by hits) ===")
display(df.head(10))

# 샘플 내용 미리보기
def show_page_snippet(page_no:int, n=600):
    matches = [d for d in docs if d.metadata.get("page")==page_no]
    if not matches:
        print("해당 페이지 없음:", page_no); return
    txt = (matches[0].page_content or "").strip().replace("\n"," ")
    print(f"[p.{page_no}] {txt[:n]}{'...' if len(txt)>n else ''}")

print("\n예시) show_page_snippet(0)  # 원하는 페이지 번호로 바꿔 확인")
